In [1]:
import pandas as pd
import redditcleaner
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import FreqDist
import emoji
import re
import en_core_web_sm
import spacy
nltk.download('wordnet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('vader_lexicon')
from numpy import *
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Austin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Austin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
#gather all data from csv, prolly really inefficient
new_data_1 = pd.read_csv('./collected_data/combined_csv.csv', low_memory=False)
old_data_2 = pd.read_csv('./cleaned_dataset.csv', low_memory=False)


subreddit_list_1 = ["JoeRogan", "walkaway", "ukraine", "conspiracy", "Coronavirus", 
                  "antiwork", "politics", "Conservative", "progressive", "Libertarian", "AskThe_Donald"]

subreddit_list_2 = ["JoeRogan", "walkaway", "ukraine", "WorkReform", "Coronavirus", 
                  "antiwork", "politics", "Conservative", "progressive", "Libertarian", "HermanCainAward"]

In [3]:
# opening the file in read mode
my_file = open("./guh.txt", "r")
  
# reading the file
keyword_dict = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
keyword_dict_list = keyword_dict.split("\n")
for i in range(len(keyword_dict_list)):
    keyword_dict_list[i] = keyword_dict_list[i].lower()
# print(keyword_dict_list)
my_file.close()

In [4]:
def get_searched_word_df(df, searchwords):
    df['body'] = df["body"].map(str)
    df['body'] = df['body'].str.lower()
    df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
    
    contains_sw_array = []
    
    searchwords_np = np.array(searchwords)
    
    body_df = df['body']
    
    for body in body_df:
        body_split_np = np.array(body.split())
        
        mask = np.isin(searchwords_np, body_split_np)
        
        if True in mask:
            contains_sw_array.append(1)
        else:
            contains_sw_array.append(0)

    return contains_sw_array

In [5]:
array = get_searched_word_df(new_data_1, keyword_dict_list)

In [6]:
print(new_data_1.shape)
print(len(array))

print(array)


(503552, 13)
503552
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
pd.set_option('display.max_colwidth', None)
new_data_1['body'].iloc[[17]]

17    i mean yeah i’ve seen a low level of mask use in those communities, feel like it’s rarely mentioned but poor areas definitely are more anti-mask
Name: body, dtype: object

In [8]:
new_data_1['contains_a_keyword'] = array

In [9]:
new_data_1.shape


(503552, 14)

In [10]:
nlp = en_core_web_sm.load()
all_stopwords = nlp.Defaults.stop_words
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

#https://levelup.gitconnected.com/reddit-sentiment-analysis-with-python-c13062b862f6
def clean_body(string_unclean, all_stopwords, lemmatizer, stemmer):
    
    #remove emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    
    string_emojiless = emoji_pattern.sub(r'', string_unclean)
    
    #Tokenizing breaks apart every word in the string into an individual 
    #word which would then carry it’s own “positive” or “negative” sentiment 
    #based on our sentiment analyzer later
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|http\S+')
    tokenized_string = tokenizer.tokenize(string_emojiless)
    
    
    lower_string_tokenized = [word.lower() for word in tokenized_string]
#     print(lower_string_tokenized)
    
    #remove stopwords
    text = lower_string_tokenized
    tokens_wo_stopwords = [word for word in text if not word in all_stopwords]
#     print(tokens_wo_stopwords)
    
    #lemmatize words, returns root word of the actual word
    lemmatized_tokens = ([lemmatizer.lemmatize(w) for w in tokens_wo_stopwords])
#     print(lemmatized_tokens)
    
#     stem_tokens = ([stemmer.stem(s) for s in tokens_wo_stopwords])
#     print(stem_tokens)

#     print(type(lemmatized_tokens))
    return lemmatized_tokens

In [11]:
sia = SIA()

def apply_sentiment_analysis(cleaned_output, sia):
    
    results = []
    
    for sentences in cleaned_output:
        pol_score = sia.polarity_scores(sentences)
        pol_score['words'] = sentences
        results.append(pol_score)
    
    df = pd.DataFrame.from_records(results)
    
#     print(df.shape)
#     print(df.columns)
    
    if len(df.columns) == 0:
        average = 0
    else:
        df['label'] = 0
        df.loc[df['compound'] > .10, 'label'] = 1
        df.loc[df['compound'] < -.10, 'label'] = -1
        average = df['label'].mean()
    #     print(average)
    
    return average

In [22]:
def create_sentiment_analysis_df(df, all_stopwords, lemmatizer, stemmer, sia):
    sentiment_values_list = []
    
    count = 0
    for i in range(len(df)):
        body = df["body"].iloc[i]
        cleaned_text = clean_body(body, all_stopwords, lemmatizer, stemmer)
        ave = apply_sentiment_analysis(cleaned_text, sia)
        sentiment_values_list.append(ave)
        count = count + 1
        if (count%10000) == 0:
            print("Completed: " + str(count) + " comments.")
        
        
    return sentiment_values_list

In [17]:
# sentiment_values_list = create_sentiment_analysis_df(new_data_1, all_stopwords, lemmatizer, stemmer, sia)

In [23]:
only_with_keywords_1  = new_data_1.loc[new_data_1['contains_a_keyword'] == 1]

print(only_with_keywords_1.shape)
print(new_data_1.shape)



(28286, 14)
(503552, 14)


In [24]:

sentiment_values_list = create_sentiment_analysis_df(new_data_1, all_stopwords, lemmatizer, stemmer, sia)

Completed: 10000 comments.
Completed: 20000 comments.
Completed: 30000 comments.
Completed: 40000 comments.
Completed: 50000 comments.
Completed: 60000 comments.
Completed: 70000 comments.
Completed: 80000 comments.
Completed: 90000 comments.
Completed: 100000 comments.
Completed: 110000 comments.
Completed: 120000 comments.
Completed: 130000 comments.
Completed: 140000 comments.
Completed: 150000 comments.
Completed: 160000 comments.
Completed: 170000 comments.
Completed: 180000 comments.
Completed: 190000 comments.
Completed: 200000 comments.
Completed: 210000 comments.
Completed: 220000 comments.
Completed: 230000 comments.
Completed: 240000 comments.
Completed: 250000 comments.
Completed: 260000 comments.
Completed: 270000 comments.
Completed: 280000 comments.
Completed: 290000 comments.
Completed: 300000 comments.
Completed: 310000 comments.
Completed: 320000 comments.
Completed: 330000 comments.
Completed: 340000 comments.
Completed: 350000 comments.
Completed: 360000 comments.
C

In [25]:
new_data_1['sentiment_value'] = sentiment_values_list

In [26]:
new_data_1.to_csv('out.csv')  

In [27]:
new_data_1.head

<bound method NDFrame.head of                       author author_fullname author_premium  \
0             one_in_the_wtf        t2_8e8s8          False   
1               BEASTIN-25-8       t2_14vlpa          False   
2       International-Crab-9     t2_6oi6pk3x          False   
3            beenlobotomized      t2_o6bekgw          False   
4               BEASTIN-25-8       t2_14vlpa          False   
...                      ...             ...            ...   
503547     hopelessturtle301     t2_czjswurq          False   
503548            Shinikage1        t2_90xs4          False   
503549              Hallwart     t2_3iseahcc          False   
503550              aceratv6     t2_70agnxc6          False   
503551        throwawayata79     t2_7mexlkcn           True   

                                                                                                                                                                                                                     

In [35]:
df_split = np.array_split(new_data_1, 4)

In [39]:
for i in range(len(df_split)):
    print("out_" + str(i) + ".csv")
    df_split[i].to_csv("out_" + str(i) + ".csv")  

out_0.csv
out_1.csv
out_2.csv
out_3.csv


In [40]:
print(df_split[0].shape)

(125888, 15)


In [42]:
print(503552/4)

125888.0


In [43]:
new_data_1.columns

Index(['author', 'author_fullname', 'author_premium', 'body', 'created_utc',
       'id', 'link_id', 'parent_id', 'permalink', 'score', 'subreddit',
       'subreddit_id', 'total_awards_received', 'contains_a_keyword',
       'sentiment_value'],
      dtype='object')